In [ ]:
 %load_ext autoreload
%autoreload 2
"""
    Some imports used throughout the notebook
"""
import time

from visualize import *

import cpmpy
from cpmpy.transformations.normalize import toplevel_list
from factory import *
from read_data import get_data
from cpmpy.tools.explain import mus

import numpy as np
np.set_printoptions(linewidth=90)
# preload solvers
from cpmpy import SolverLookup
names = SolverLookup.solvernames()

import networkx as nx
import re
draw = lambda g,**kwargs : nx.draw_circular(g, width=5, node_size=500,**kwargs)
cmap = ["black", "yellow", "cyan", "lightgreen", "blue"]

def pinpoint_to_constraints(G, model):
    #assert model.solve()
    draw_conflict(G, mus(m.constraints))

def graph_coloring(G, max_colors=None):
    n = G.number_of_nodes()
    if max_colors is None:
        max_colors = n
    
    m = cp.Model()
    # worst case: give each node a different color
    nodes = cp.intvar(1, max_colors, shape=n, name="Node")
    # constrain edges to have differently colored nodes
    m += [nodes[n1] != nodes[n2] for n1, n2 in G.edges()]
    m.minimize(cp.max(nodes))
    
    return m, nodes

def graph_highlight(graph, cons, **kwargs):
    edges = []
    for c in cons:
        n1, n2 = c.args
        if n1.name == "max": continue
        a = int(re.search("\[[0-9]*\]", str(n1)).group()[1:-1])
        b = int(re.search("\[[0-9]*\]", str(n2)).group()[1:-1])
        edges.append((a,b))
        
    colors = ["red" if (a,b) in edges else "black" for (a,b) in graph.edges()]       
    return draw(graph, edge_color=colors, **kwargs)

def get_avg_outputlits(seq):
    lits = sum(len(out.literals()) for _,_,out in seq[:-1])
    return lits / (len(seq)-1)

def get_avg_stepsize(seq):
    return sum(len(s) for _,s,_ in seq) / len(seq)

## Explanation techniques for CP
### Prof. Tias Guns, KU Leuven, Belgium
#### In collaboration with Ignace Bleukx, Emilio Gamba, Bart Bogaerts, Jo Devriendt, Dimos Tsouros

<p>&nbsp;</p>

<table><tr style="background: white;">
    <td>&nbsp;</td>
    <td style="text-align: center; vertical-align: middle;"><img src="img/kul.jpg" width=40%></td>
    <td style="text-align: center; vertical-align: middle;"><img src="img/erc.jpg" width=45%></td>
</tr></table>

<!-- Thanks to Bart Bogaerts, Emilio Gamba and Jo Devriendt -->


<small>This presentation is an executable Jupyter notebook</small>

Link to slides and more examples: https://github.com/CPMpy/XCP-explain

## AI and Constraint Solving

<center><img src="img/intro_ai.png" width="65%" align="center" style="margin-top:10px"></center>


## Constraint Solving

<img src="img/solutions_vizual.png" width="45%" align="right" style="margin-top:100px">

Solving combinatorial optimization problems in AI

- Vehicle Routing

- Scheduling

- Manufacturing

- Other combinatorial problems ...

## Model + Solve

<center><img src="img/model_solve.png" width=70%></center>

- Reason over the constraints to find a solution
- Find the optimal solution according to the objective 
    - (maximize gain or minimize penalty)

## What happens after Model + Solve?

<center><img src="img/model_solve.png" width=70%></center>

- What if no solution?
- What if user does not _like_ the solution?
- What if the user _expected_ a different solution?
- ...

Need for **explanations**

## Explainable Constraint Programming (XCP)

In general, "**Why X?**" &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (with X a solution or UNSAT)

<img src="img/interaction_figure4.png" width=30% align="right">

- **Deductive explanation:**
  - _What causes X?_
- **Counterfactual explanation**:
  - _What if I want Y instead of X?_


<!-- Can be combined in interactive systems:
  - _Iteratively refine explanation & model_ -->

# Example XCP interaction

Toy example, graph coloring:

color each node such that no adjacent nodes have the same color

In [ ]:
G = nx.fast_gnp_random_graph(5, 0.8, seed=0)
draw(G)

# Example XCP interaction

## Lets color this graph...

In [ ]:
m, nodes = graph_coloring(G, max_colors=None)
if m.solve():
    print(m.status())
    print(f"Found optimal coloring with {m.objective_value()} colors")
    draw(G, node_color=[cmap[n.value()] for n in nodes])
else:
    print("No solution found.")

# Example XCP interaction

## Lets color this graph...

In [ ]:
draw(G, node_color=[cmap[n.value()] for n in nodes])

<img src="img/why.png" width="15%" align="left" style="margin-top:-40px; margin-right:50px; margin-left: 300px">
yes... but why do we need 4? 

# Example XCP interaction

## _Why_ do we need 4 colors?

Deductive explanation: pinpoint to constraints _causing_ this fact

In [ ]:
m, nodes = graph_coloring(G, max_colors=3) # less than 4?
if not m.solve():
    conflict = cpmpy.tools.explain.mus(m.constraints)  # Minimal Unsatisfiable Subset
    print("UNSAT is caused by the following constraints:")
    graph_highlight(G, conflict)

# Example XCP interaction

## _Why_ do we need 4 colors?

Counterfactual explanation: pinpoint to constraint *changes* that would allow, e.g. 3 colors

In [ ]:
m, nodes = graph_coloring(G, max_colors=3) # less than 4?
if not m.solve():    
    corr = cpmpy.tools.explain.mcs(m.constraints)  # Minimal Correction Subset
    print("UNSAT can be resolved by removing the following constraints:")
    graph_highlight(G, corr)

In [ ]:
# compute and visualise counter-factual solution
m2 = cp.Model([c for c in m.constraints if c not in corr])
m2.solve()
graph_highlight(G, corr, node_color=[cmap[n.value()] for n in nodes])

# Explanation techniques in the wild

<center><img src="img/app_explanations.png"  style="max-width: 80%;"></center>

## CPMpy: http://cpmpy.readthedocs.io

We will use the CPMpy modeling library in Python for this presentation

<center><img src="img/cpmpy-intro.png" style="max-width: 70%;" /></center>


## Running example in this talk: Nurse Scheduling

* The assignment of _shifts_ and _holidays_ to nurses.
* Each nurse has their own restrictions and preferences,
    as does the hospital.

### hospital constraints:

<img src="img/nurse_rost_prob.jpg" align="right">

* nb of shifts required on each day
* max nb of shifts per nurse
* max nb of weekend shifts
* min/max consecutive shifts

### nurse constraints / preferences:

* specific days off-duty
* specific shift requests (on/off)

## Running example in this talk: Nurse Scheduling

* The assignment of _shifts_ and _holidays_ to nurses.
* Each nurse has their own restrictions and preferences,
    as does the hospital.

In [ ]:
instance = "Benchmarks/Instance1.txt"  # "http://www.schedulingbenchmarks.org/nrp/data/Instance1.txt"
data = get_data(instance)

## Nurse Rostering: data

Instances from http://www.schedulingbenchmarks.org/

"benchmark test instances from various sources including industrial collaborators and scientific publications."

<!-- 7 types of hospital constraints, 2 types of nurse constraints -->

In [ ]:
#instance = "http://www.schedulingbenchmarks.org/nrp/data/Instance1.txt"
instance = "Benchmarks/Instance1.txt"
data = get_data(instance)

factory = NurseSchedulingFactory(data)
model, nurse_view = factory.get_full_model()  # CPMpy model with all constraints

model.solve() 
visualize(nurse_view.value(), factory)  # live decorated dataframe

In [ ]:
print("Nr of days to schedule:", data.horizon)
print("Nr of shift types:", len(data.shifts))

pd.merge(data.days_off, data.staff[["# ID","name"]], left_on="EmployeeID", right_on="# ID", how="left")

# Outline of the talk

## Part 1: Deductive explanations (What causes X?)

* UNSAT: minimal unsatisfiable subsets &#9194;
  - efficient MUSes
  - preferred MUSes
* SAT: explaining logical consequences
* OPT: explaining that no better solution exists

## Part 2: Counterfactual explanation (What if Y instead of X?)

* UNSAT: minimum correction subsets
* UNSAT: corrective actions
* SAT: checking a foil
* OPT: correcting the objective function
    

## Deductive Explanations for UNSAT problems

In [ ]:
# decision model, add all nurse preferences as hard constraints
factory = NurseSchedulingFactory(data)
model, nurse_view = factory.get_decision_model()
model.solve()

... no solution found

In [ ]:
constraints = toplevel_list(model.constraints, merge_and=False) # normalization for later
print(f"Model has {len(constraints)} constraints:")
for cons in constraints: print("-", cons)

## Converting to Decision model


<img src="img/explain_unsat.png" width="15%" align="left" style="margin:50px;">

* Model nurse rostering problem as decision problem <br>
    (no objective)
        
* Nurse **preferences** are also hard constraints

## Deductive Explanations for UNSAT problems

The set of all constraints is unsatisfiable.

<img src="img/allcons.png" width="20%" float="right" style="margin:50px;">


But do all constraints contribute to this?

## Deductive Explanations for UNSAT problems

### Minimal Unsatisfiable Subset (MUS)

<img src="img/mus.png" width="20%" align="left" style="margin:50px;">

Pinpoint to constraints causing a conflict

... trim model to minimal set of constraints

... minimize cognitive burden for user


### How to compute a MUS?

Deletion-based MUS algorithm

_[Joao Marques-Silva. Minimal Unsatisfiability: Models, Algorithms and Applications. ISMVL 2010. pp. 9-14]_

In [ ]:
def mus_naive(constraints):
    m = cp.Model(constraints)
    assert m.solve() is False, "Model should be UNSAT"
    
    core = constraints
    i = 0
    while i < len(core):
        subcore = core[:i] + core[i+1:]  # try all but constraint 'i'
        if cp.Model(subcore).solve() is True:
            i += 1  # removing 'i' makes it SAT, need to keep for UNSAT
        else:
            core = subcore #  can safely delete 'i'
    return core

### How to compute a MUS, <u>efficiently</u>?

In [ ]:
t0 = time.time()
core = mus_naive(model.constraints)
print(f"Naive MUS took {time.time()-t0} seconds")

In [ ]:
t0 = time.time()
core = cpmpy.tools.explain.mus(model.constraints)
print(f"Assumption-based MUS took {time.time()-t0} seconds")

### How to compute a MUS, <u>efficiently</u>?



In [ ]:
def mus_assum(constraints, solver="ortools"):
    # add indicator variable per constraint
    constraints = toplevel_list(constraints, merge_and=False)
    assump = cp.boolvar(shape=len(constraints), name="assump")  # Boolean indicators
    m = cp.Model(assump.implies(constraints))  # [assump[i] -> constraints[i] for all i]
    
    s = cp.SolverLookup.get(solver, model)
    assert s.solve(assumptions=assump) is False, "Model should be UNSAT"

    core = s.get_core()  # start from solver's UNSAT core
    i = 0
    while i < len(core):
        subcore = core[:i] + core[i+1:]  # try all but constraint 'i'
        if s.solve(assumptions=subcore) is True:
            i += 1  # removing 'i' makes it SAT, need to keep for UNSAT
        else:
            core = subcore
    return core

### How to compute a MUS, <u>efficiently</u>?

<center><img src="img/mus_assum.png" style="max-width: 85%;" /></center>

## Deepdive: incremental CDCL solving with assumption variables 1/4

<center><img src="img/slide_cdcl1.png" style="max-width: 85%;" /></center>

## Deepdive: incremental CDCL solving with assumption variables 2/4

<center><img src="img/slide_cdcl2.png" style="max-width: 85%;" /></center>

## Deepdive: incremental CDCL solving with assumption variables 3/4

<center><img src="img/slide_cdcl3.png" style="max-width: 85%;" /></center>

## Deepdive: incremental CDCL solving with assumption variables 4/4

<center><img src="img/slide_cdcl4.png" style="max-width: 85%;" /></center>

### How to compute a MUS, <u>efficiently</u>? (recap after deepdive)

<center><img src="img/mus_assum.png" style="max-width: 85%;" /></center>

### How to compute a MUS, <u>efficiently</u>?

Assumption-based incremental solving only for Boolean SAT problems?

<b>No!</b>

* CP solvers: <i>Lazy Clause Generation</i> (e.g. OrTools)
* Pseudo-Boolean solvers: <i>Conflict-Driven Cutting Plane Learning</i> (e.g. Exact)
* SMT solvers: <i>SAT Module Theories with CDCL</i> (e.g. Z3)
* MaxSAT solvers: <i>Core-guided solvers</i>

## Deductive Explanations for UNSAT problems

A MUS is a deductive explanation of UNSAT:

these constraints minimally entail failure


In [ ]:
subset = cpmpy.tools.explain.mus(model.constraints)
print("Length of MUS:", len(subset))
for cons in subset: print("-", cons)

In [ ]:
visualize_constraints(subset, nurse_view, factory)

### Many MUS'es may exist...

_Liffiton, M.H., & Malik, A. (2013). Enumerating infeasibility: Finding multiple MUSes quickly. In
Proceedings of the 10th International Conference on Integration of AI and OR Techniques in Constraint
    Programming (CPAIOR 2013) (pp. 160–175)_

In [ ]:
# MARCO MUS/MSS enumeration
from explanations.marco_mcs_mus import do_marco
solver = "ortools"  # default solver
if "exact" in cp.SolverLookup.solvernames(): solver = "exact"  # fast for increment solving
    
t0 = time.time()
cnt = 0
for (kind, sset) in do_marco(model, solver=solver):
    if kind == "MUS":
        print("M", end="")
        cnt += 1
    else: print(".", end="") # MSS
    
    if time.time() - t0 > 15:  break  # for this presentation: break after 15s
print(f"\nFound {cnt} MUSes in", time.time() - t0)
 

### Many MUS'es may exist...

<img src="img/musses.png" width="40%" align="left" style="margin-left:50px; margin-right:50px">

This problem has just 168 constraints, yet 100.000+ MUSes exist...

Which one to show? 

In explanations less is more, so lets find the **smallest one directly!**

## Influencing which MUS is found?

<b>QuickXPlain algorithm</b> <i>(Junker, 2004)</i>.
Widely used, in model-based diagnosis, recommender systems, verification, and more.

Divide-and-conquer given a lexicographic order over the constraints:


<center><img src="img/quickxplain.png" style="max-width: 70%;" /></center>

## Influencing which MUS is found?

<b>QuickXPlain algorithm</b> <i>(Junker, 2004)</i>.
Widely used, in model-based diagnosis, recommender systems, verification, and more.

Divide-and-conquer given a lexicographic order over the constraints

In [ ]:
def define_order(cons):
    if "day off" in str(cons):
        return 100 # we do not want to mess with vacation days
    return 10 # else

constraints = toplevel_list(model.constraints, merge_and=False)
constraints = sorted(constraints, key=define_order)

# find core with quickXplain
from cpmpy.tools.mus import quickxplain

prefered_subset = quickxplain(constraints, solver="exact")
for cons in prefered_subset: print("-",cons)

In [ ]:
subset = cpmpy.tools.explain.quickxplain(sorted(model.constraints, key=lambda c: -len(c)), solver="exact")
print("ordering '-len': Length of MUS:", len(subset))
subset = cpmpy.tools.explain.quickxplain(sorted(model.constraints, key=lambda c: len(c)), solver="exact")
print("ordering 'len': Length of MUS:", len(subset))

In [ ]:
visualize_constraints(subset, nurse_view, factory)

## Optimising which MUS is found?

OUS: Optimal Unsatisfiable Subsets <i>(Gamba, Bogaerts, Guns, 2021)</i>. Every constraints has a weight.

Implicit Hitting-Set Algorithm:

1. If a subset is SAT, can <i>grow</i> it to a Maximal Satisfiable Subset (MSS)
2. The complement of a MSS is a Minimum Correction Subset (MCS)
3. Theorem: A MUS is a hitting set of the MCSes

<table><tr>
    <td width=10%>
        <center><img width=60% src="img/mcs.png" /></center>
    </td> 
    <td width=10%>
        <center><img width=60% src="img/hittingset.png" /></center>
    </td>
</tr></table>

## Optimising which MUS is found?

OUS: Optimal Unsatisfiable Subsets <i>(Gamba, Bogaerts, Guns, 2021)</i>. Every constraints has a weight.

1. Initialize sets-to-hit $\mathcal{H}$ (e.g. insert set of all constraints)
2. Find *optimal* hitting set $S$ and check if UNSAT
3. If SAT: grow and take complement = MCS $K$, add to sets-to-hit $\mathcal{H}$
4. Repeat until UNSAT: optimal unsatisfiable subset $S$ found

<center><img src="img/smus.png" width=55% /></center>

## <u>Efficiently</u> optimising which MUS is found?

OUS: Optimal Unsatisfiable Subsets <i>(Gamba, Bogaerts, Guns, 2021)</i>. Every constraints has a weight.

&nbsp;

<center><img src="img/smus_efficient.png" width=65%/></center>

## Optimising which MUS is found?

OUS: Optimal Unsatisfiable Subsets <i>(Gamba, Bogaerts, Guns, 2021)</i>. Every constraints has a weight.

In [ ]:
from explanations.subset import omus  # not (yet) part of CPMpy

small_subset = omus(model.constraints, weights=1, solver="exact", hs_solver="ortools")

print("Length of OUS:", len(small_subset))
for cons in small_subset:  
    print("-", cons)

In [ ]:
visualize_constraints(small_subset, nurse_view, factory)

# Outline of the talk

## Part 1: Deductive explanations (What causes X?)

* UNSAT: minimal unsatisfiable subsets
  - efficient MUSes
  - preferred MUSes
* SAT: explaining logical consequences &#9194;
* OPT: explaining that no better solution exists

## Part 2: Counterfactual explanation (What if Y instead of X?)

* UNSAT: minimum correction subsets
* UNSAT: corrective actions
* SAT: checking a foil
* OPT: correcting the objective function
    

## Deductive Explanations for SAT problems

### Explaining logical consequences

<u>Logical consequence</u>: a variable assignment entailed by the constraints and the current partial assignment

<u>Maximal consequence</u>: precision- maximal partial assignment

* Maximal consequence = intersection of all possible solutions
* If solution is unique, maximal consequence = unique solution

### TODO: Simple algorithm of max-propagate? (not priority)

## Deductive Explanations for SAT problems

A framework for step-wise explaining how to solve constraint satisfaction problems <i>(Bogaerts, Gamba, Guns, 2021)</i>

&nbsp;

<center><img src="img/maxconsequence.png" width=75%/></center>

## Deductive Explanations for SAT problems

A framework for step-wise explaining how to solve constraint satisfaction problems <i>(Bogaerts, Gamba, Guns, 2021)</i>

&nbsp;

<center><img src="img/stepwise.png" /></center>

## Deductive Explanations for SAT problems

A framework for step-wise explaining how to solve constraint satisfaction problems <i>(Bogaerts, Gamba, Guns, 2021)</i>

<table><tr style="background: white;">
    <td>&nbsp;</td>
    <td style="text-align: center; vertical-align: middle;"><img src="img/onestep_example.png" width=90%></td>
    <td style="text-align: center; vertical-align: middle;"><img src="img/onestep_expl.png" width=80%></td>
</tr></table>

## Deductive Explanations for SAT problems

A framework for step-wise explaining how to solve constraint satisfaction problems <i>(Bogaerts, Gamba, Guns, 2021)</i>

<center><img src="img/onestep_mus.png" width=50%></center>

## Deductive Explanations for SAT problems

A framework for step-wise explaining how to solve constraint satisfaction problems <i>(Bogaerts, Gamba, Guns, 2021)</i>

# TODO!! Help...

In [ ]:
# do CP propagate (the one of CPMpy main?) on nurse rostering satisfaction variant?
# if it contains an assigned variable, take (one) of it and formulate as an OUS?
# call OUS and display?
# then again for another one? (optional)

# Outline of the talk

## Part 1: Deductive explanations (What causes X?)

* UNSAT: unsatisfiable subsets
  - Minimal Unsatisfiable Subsets
  - MUSX naive algorithm
  - Efficient MUSes with incremental solvers
    - Assumption variables in clause learning solvers
    - Unsat core extraction
  - MUSX in practice (incremental + ordering of variables)
  - Many MUSes (enumeration with Marco)
  - Preferences over MUSes: QuickXplain algorithm
  - Optimal unsatisfiable subsets: OUS
    - hitting set algorithm
    - in practice (objective incrementality HS, subset incrementality SAT, ?complementary MCSs?)
* SAT: explaining logical consequences
  - computing (all) logical consequences:
      - unique solution
      - propagated assignment
      - maximal consequence algorithm
  - explaining one logical consequence
      - reformulation to MUS/OUS
  - step-wise explaining all logical consequences
      - OCUS
* OPT: explaining that no better solution exists
  - reformulation to MUS/OUS

## Part 2: Counterfactual explanation (What if Y instead of X?)

* UNSAT: Minimum Correction Subsets
  - graph coloring example
  - MCS / MSS relation
  - Optimisation over indicator variables
* UNSAT: corrective actions
  - Slack variables
  - Optimisation over slack variables
* SAT: checking a foil
  - concept of a 'foil'
  - add foil, recompute (explain UNSAT if needed)
* OPT: correcting the objective function
  - inverse optimisation
    

## Deductive explanations with MUS - conclusions

- MUS'es can poinpoint the user to a conflict, helps to understand

BUT

- There can be many MUSes, which one to show? Smallest? _Learn_ preferences?
- No guarantee the smallest MUS is understandable
    - Worst case, all constraints in conflict! (e.g., pigeon-hole problem)
- **MUSes do not show interaction between constraints**

## Step-wise explanation

<img src="img/explain_step-wise.png" width="20%" align="right" style="margin:50px;">

- Disect the conflict into smaller steps 
- Show _interaction_ between constraints <br>
  -> Step-wise Explanations

> Ignace Bleukx, Jo Devriendt, Emilio Gamba, Bart Bogaerts, Tias Guns. Simplifying Step-wise Explanation Sequences. 29th International Conference on Principles and Practice of Constraint Programming (CP23), 2023.

## Step-wise explanations

Find a sequence of small derivation steps, _explaining_ a goal set of literals (or a goal _fact_)

Each _step_ consists of a tuple $(\mathcal{E}, \mathcal{S}, \mathcal{N})$


Current work: explain how to solve logic puzzles **with a single solution** 

> Bogaerts, Bart, Emilio Gamba, and Tias Guns. "A framework for step-wise explaining how to solve constraint satisfaction problems." Artificial Intelligence 300 (2021): 103550.

<img src="img/stepwise.png" width="100%" align="center" style="">


In that setting _no_ risk on redundancy as **every** literal (cell in the sudoku) needs to be explained

Imagine only a subset of the cells needs to be explained.

How to decide which cells to derive during the sequence? Maybe "uninteresting cells" to the user are usefull as intermediate result?

Also applies to the UNSAT case, on which we will focus for now

## Step wise explaining unsatisfiability

**Challenges**

- How to find interpretable sequences? 
    - _I.e., with few and small steps?_
- How to deal with redundancy in the sequence? 
    - _I.e., how to decide what information is relevant to derive?_
- How to keep the algorithm interactive?
    - _I.e., how to find good sequences fast?_

## 3-phase approach:

Greedy construction of the sequence:
<img src="img/manysteps.png" width="80%" style="margin-bottom: -150px; margin-top: -50px"><br>
Removing redundant steps:
<img src="img/littledirtysteps.png" width="80%" style="margin-bottom: -100px;  margin-top:-50px;">
Clean each step from spurious input/output:
 <img src="img/littlecleansteps.png" width="45%" style="margin-top:-120px">

###  1. Greedy construction

<img src="img/greedy_construct.png" width="50%" align="left" style="margin:20px;">

**Maximal sequences**: 
- Use everyting available
- Derive everything possible



## Greedy construction for the Nurse rostering problem

In [ ]:
from explanations.stepwise import forward_construction
        
seq = forward_construction(model.constraints)
print(f"Found sequence of length {len(seq)}")
print("Avg nb of constraints used:", get_avg_stepsize(seq))

In [ ]:
nurse_view.clear()
for step in seq:
    display(visualize_step(step, nurse_view, factory))

\+ The good: <br>
- we get a sequence of explanation steps leading to the conflict!
- each step has low number of constraints

\- The bad: <br>
- the sequence is VERY long...

### 2. Filtering redundant steps

<img src="img/deletion_filter.png" width="40%" align="left" style="margin-right:40px;">
Try to remove step from sequence and check if sequence is valid

- If yes: remove the step,
- If no: we need the step in the sequence

Produces **maximal sequences**

In [ ]:
from explanations.stepwise import backward_filtering

filtered = backward_filtering(model.constraints, seq)
print(f"Filtered sequence to length {len(filtered)}")
print("Avg nb of output literals:", get_avg_outputlits(filtered))

In [ ]:
nurse_view.clear()
for step in filtered:
    display(visualize_step(step, nurse_view, factory))

\+ The good: <br>
- Our sequence is much shorter and easier to comprehend

\- The bad: <br>
- Individual steps may derive or use too much information

### 3. Simplify steps by removig input/output information

<img src="img/relaxation.png" width="50%" align="left" style="margin-right:40px;">

In [ ]:
from explanations.stepwise import relax_sequence

relaxed = relax_sequence(filtered, time_limit=100)
print(f"Relaxed sequence has length {len(relaxed)}")
print("Avg nb of output literals:", get_avg_outputlits(relaxed))

In [ ]:
nurse_view.clear()
for step in relaxed:
    display(visualize_step(step, nurse_view, factory))

## Results on benchmarks

<img src="img/steps.png" width="80%">

<img src="img/inputlits.png" width="80%">

<img src="img/outputlits.png" width="80%">

<img src="img/sudoku_runtime.pdf" width=44.5% style="float:left">
<img src="img/jobshop_runtime.pdf" width="40%" style="float:left;margin-top:25px">

# Conclusions

- Deductive explanations are a helpful means of explaining _why_ a problem is unsatisfiable

- Step-wise explanations can be applied to the problem of explaining unsatisfiability, BUT:
    - Need to take care for redundancy
- Open questions:
    - _Which_ sequence to show? _The_ shortest one? _Learn_ preferences?
    - How to improve runtime of the algorithm?
    - Tighter integration of forward-construction and backwards-filtering?
    
    




# Want to learn more?

<img src="img/qr-code.png" align="right" width="30%" style="margin-top:20px">

Tutorial with variety of explanation techniques recorded

Also includes _how_ to adapt the model to make it satisfiable 

(i.e., _contrastive_ explanations)

